In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import pandas as pd
import psycopg2
import warnings
import joblib
import h2o
import time

In [21]:
pip list

Note: you may need to restart the kernel to use updated packages.
Package                      Version   Editable project location
---------------------------- --------- -------------------------------------------------------------
absl-py                      1.4.0
anyio                        3.7.1
asgiref                      3.7.2
asttokens                    2.2.1
astunparse                   1.6.3
awscli                       1.29.36
backcall                     0.2.0
Bashutils                    0.0.4
bcrypt                       4.0.1
blinker                      1.6.3
blis                         0.7.9
boto3                        1.29.6
botocore                     1.32.6
cachetools                   5.3.1
catalogue                    2.0.8
catboost                     1.2.2
certifi                      2023.5.7
cffi                         1.15.1
charset-normalizer           3.1.0
click                        7.1.2
cloudpickle                  3.0.0
cmdstanpy                

In [2]:
warnings.simplefilter("ignore")
localH2o = h2o.init(nthreads = -1)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.391-b13, mixed mode)
  Starting server from C:\Users\milen\AppData\Local\Programs\Python\Python310\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\milen\AppData\Local\Temp\tmpk0jzs858
  JVM stdout: C:\Users\milen\AppData\Local\Temp\tmpk0jzs858\h2o_milen_started_from_python.out
  JVM stderr: C:\Users\milen\AppData\Local\Temp\tmpk0jzs858\h2o_milen_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.1
H2O_cluster_version_age:,"1 year, 4 months and 5 days !!!"
H2O_cluster_name:,H2O_from_python_milen_dyd6o2
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.481 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [3]:
def get_best_models(area, quant):
    try:
        connection = psycopg2.connect(
            database="testy",
            user="postgres",
            password="SENHA",
            host="127.0.0.1",
            port="5432"
        )

        # Cria um cursor para executar consultas
        cursor = connection.cursor()

        # Executa a consulta
        query = f"""
                SELECT "model", "algorithm", "accuracy", "precision", "recall", "f1-score" FROM public.metrics
                WHERE "area" = '{area}'
                ORDER BY "f1-score" DESC LIMIT {quant};
                """
        cursor.execute(query)

        # Recupera os resultados da consulta como uma lista de tuplas
        results = cursor.fetchall()

        # Cria um DataFrame pandas com os resultados
        df_modelos = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])

        # Fecha o cursor e a conexão
        cursor.close()

    except psycopg2.Error as error:
        print("Erro ao conectar ao PostgreSQL:", error)

    finally:
        # Fecha a conexão com o banco de dados
        if 'connection' in locals():
            connection.close()
        
        return df_modelos

In [4]:
def get_prevision(row, entrada):
    modelo = row[1]
    print('C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/' + row[0])

    if modelo == "Random Forest" :
        resultado = prevRF(entrada, ('C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/' + row[0]))

    if modelo == 'AdaBoost':
        resultado = prevADA(entrada, ('C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/' + row[0]))

    if modelo == 'XGBoost':
        resultado = prevXGB(entrada, ('C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/' + row[0]))

    if modelo == 'CatBoost':
        resultado = prevCAT(entrada, ('C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/' + row[0]))

    if modelo == 'Decision Tree':
        resultado = prevDT(entrada, ('C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/' + row[0]))

    if modelo == 'GaussianNB':
        resultado = prevGNB(entrada, ('C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/' + row[0]))

    if modelo == 'Logistic Regression':
        resultado = prevLOG(entrada, ('C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/' + row[0]))
        
    if modelo == 'SVC':
        resultado = prevSVC(entrada, ('C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/' + row[0]))

    return resultado

In [5]:
def prevH2O(abstract):
    model = h2o.load_model('C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/DeepLearning_model_R_1670582405235_1')
    prev = model.predict(h2o.H2OFrame(abstract))
    del(model)
    return prev[0, 0]

In [6]:
def prevXGB(abstract, path_model):
    model, encoder, _, _ = joblib.load(path_model)
    
    prev = model.predict(abstract)

    label_encoder = LabelEncoder()
    label_encoder.fit_transform(encoder)
    
    prev = label_encoder.inverse_transform(prev)
    del(model, encoder)
    return prev[0]

In [7]:
def prevADA(abstract, path_model):
    model, encoder, _, _ = joblib.load(path_model)
    
    prev = model.predict(abstract)

    label_encoder = LabelEncoder()
    label_encoder.fit_transform(encoder)
    
    prev = label_encoder.inverse_transform(prev)
    del(model, encoder)
    return prev[0]

In [8]:
def prevCAT(abstract, path_model):
    model, _= joblib.load(path_model)
    
    prev = model.predict(abstract)
    del(model)
    return prev[0][0]

In [9]:
def prevDT(abstract, path_model):
    model, encoder, _, _ = joblib.load(path_model)
    
    prev = model.predict(abstract)

    label_encoder = LabelEncoder()
    label_encoder.fit_transform(encoder)
    
    prev = label_encoder.inverse_transform(prev)
    del(model, encoder)
    return prev[0]

In [10]:
def prevGNB(abstract, path_model):
    model, encoder, _ = joblib.load(path_model)
    
    prev = model.predict(abstract)

    label_encoder = LabelEncoder()
    label_encoder.fit_transform(encoder)
    
    prev = label_encoder.inverse_transform(prev)
    del(model, encoder)
    return prev[0]

In [11]:
def prevLOG(abstract, path_model):
    model, encoder, _, _ = joblib.load(path_model)
    
    prev = model.predict(abstract)

    label_encoder = LabelEncoder()
    label_encoder.fit_transform(encoder)
    
    prev = label_encoder.inverse_transform(prev)
    del(model, encoder)
    return prev[0]

In [12]:
def prevSVC(abstract, path_model):
    model, encoder, _, _ = joblib.load(path_model)
    
    prev = model.predict(abstract)

    label_encoder = LabelEncoder()
    label_encoder.fit_transform(encoder)
    
    prev = label_encoder.inverse_transform(prev)
    del(model, encoder)
    return prev[0]

In [13]:
def prevRF(abstract, path_model):
    model, _= joblib.load(path_model)
    
    prev = model.predict(abstract)
    del(model)
    return prev[0]

In [14]:
def prev(entrada, qnt_models = 5):
    start = time.time()
    
    area = prevH2O(entrada)
    
    models = get_best_models(area, min(qnt_models, 8))
    
    results = []
    
    for index, row in models.iterrows():
        results.append(get_prevision(row, entrada))
    

    map = {}

    for i, result in enumerate(results):
        map[result] = map.get(result, 0) + models['f1-score'][i]
        
    sub, max_value = max(map.items(), key=lambda x: x[1])

    ans = [f"""c("{sub}", "{area}")"""]
    
    ans.append(results)
    
    end = time.time()
    print(f"""tempo total para previsão {end - start}s""")
    
    return ans

In [15]:
path_df = 'C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/models/df.100x1x100.Ocorrencias.csv'
df_id = 'eef474adc4c2d494dca53fa6b3bd8211'

In [16]:
df = pd.read_csv(path_df)

In [17]:
df.head(1)

,Status,v1075,v592,v1341,v2630,v101,v292,v5921,v42,v1349,...,v12657,v1222,v14992,v2112,v6086,v22018,v3773,v4492,v3677,v9307
0,"c(""BIOLOGIA ANIMAL"", ""CIENCIAS BIOLOGICAS"")",0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
entrada = df.tail(1)
entrada = entrada.drop(['Status'], axis=1)

In [19]:
entrada

,v1075,v592,v1341,v2630,v101,v292,v5921,v42,v1349,v934,...,v12657,v1222,v14992,v2112,v6086,v22018,v3773,v4492,v3677,v9307
173356,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
prev(entrada)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/Engenharias_rf_comp.pkl
C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/Engenharias_xgboost_comp.pkl
C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/Engenharias_svc_comp.pkl
C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/Engenharias_catboost_comp.pkl
C:/Users/milen/OneDrive/Documentos/GitHub/Chat_ibict/Progressao/static/Modelos/Engenharias_logistic_regression_comp.pkl
tempo total para previsão 5.589484930038452s


['c("ENGENHARIA DE PRODUCAO", "ENGENHARIAS")',
 ['ENGENHARIA DE PRODUCAO',
  'ENGENHARIA DE PRODUCAO',
  'ENGENHARIA DE PRODUCAO',
  'ENGENHARIA DE PRODUCAO',
  'ENGENHARIA DE PRODUCAO']]